In [7]:
target_file = 'shiraishi.jpg'
#img_dir = os.path.abspath(os.path.dirname("__file__")) + '/images/'
img_size = (200, 200)


target_img_path = "./images/" + target_file
target_img = cv2.imread(target_img_path, cv2.IMREAD_GRAYSCALE)
target_img = cv2.resize(target_img, img_size)

bf = cv2.BFMatcher(cv2.NORM_HAMMING)
#detector = cv2.ORB_create()
detector = cv2.AKAZE_create()
(target_kp, target_des) = detector.detectAndCompute(target_img, None)

print("抽出方法：AKAZE")
print('TARGET: %s' % (target_file))

files = os.listdir("./images/")
for file in files:
    if file == '.DS_Store' or file == target_file:
        continue

    comparing_img_path = "./images/" + file
    try:
        comparing_img = cv2.imread(comparing_img_path, cv2.IMREAD_GRAYSCALE)
        comparing_img = cv2.resize(comparing_img, img_size)
        (comparing_kp, comparing_des) = detector.detectAndCompute(comparing_img, None)
        matches = bf.match(target_des, comparing_des)
        dist = [m.distance for m in matches]
        ret = sum(dist) / len(dist)
    except cv2.error:
        ret = 100000

    print(file, ret)

抽出方法：AKAZE
TARGET: shiraishi.jpg
akimoto.png 214.3125
akimoto2.jpg 158.8125
akimoto3.jpg 143.3125
akimoto4.jpg 124.9375
akimoto5.jpg 138.1875
ikuta.jpg 100.875
karin.png 124.125
manatsu.jpg 131.0
manatsu2.png 127.125
manatsu_eyes.jpg 163.25
nishino.png 163.9375
nyanchu.png 117.3125
nyanchu_eyes.png 116.8125
sonano.jpg 138.75
two1.jpg 151.25
two2.jpg 132.625
twoshot_akimoto.jpg 138.375
twoshot_unagi.jpg 131.875
unagi.png 122.3125
unagi2.jpg 136.625
unagi3.jpg 148.4375
unagi4.jpg 179.4375


In [2]:
import cv2
import os
import numpy as np
from PIL import Image

#トレーニング画像
train_path = './train_images'

#テスト画像
test_path = './test_images'

#Haar-like特徴分類器
cascadePath = "/Users/daichi/Downloads/opencv-3.4.1/opencv-3.4.1/data/haarcascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

#FisherFace
#recognizer = cv2.face_FisherFaceRecognizer.create()
recognizer = cv2.face_LBPHFaceRecognizer.create()

#pathしたフォルダ内の画像を習得
def get_images_and_labels(path):
    #画像を格納する配列
    images = []
    #ラベルを格納する配列
    labels = []
    #ファイル名を格納する配列
    files = []
    for f in os.listdir(path):
        #画像のパス
        image_path = os.path.join(path, f)
        #グレースケールで読み込み
        image_pil = Image.open(image_path).convert('L')
        #Numpyの配列に格納
        image = np.array(image_pil, 'uint8')
        #Haar-like特徴分類器で顔を検知
        faces = faceCascade.detectMultiScale(image)
        #検出した画像の処理
        for(x, y, w, h) in faces:
            #200×200にリサイズ
            roi = cv2.resize(image[y: y + h, x: x + w], (200, 200), interpolation=cv2.INTER_LINEAR)
            #画像を配列に格納
            images.append(roi)
            #ファイル名からラベルを取得
            labels.append(int(f[7:9]))
            #ファイル名を配列に格納
            files.append(f)
    
    return images, labels, files

#トレーニング画像を取得
images, labels, files = get_images_and_labels(train_path)

#トレーニング実施
recognizer.train(images, np.array(labels))

#テスト画像を取得
test_images, test_labels, test_files = get_images_and_labels(test_path)

i=0
while i < len(test_labels):
    #テスト画像に対して予測実施
    label, confidence = recognizer.predict(test_images[i])
    #予測結果をコンソール出力
    print("Test Image: {}, Predicted Label: {}, Confidence: {}".format(test_files[i], label, confidence))
    #テスト画像表示
    cv2.imshow("test image", test_images[i])
    cv2.waitKey(300)
    i += 1

#終了処理
cv2.destroyAllWindows()

Test Image: subject01.akimoto.jpg, Predicted Label: 1, Confidence: 21.786044627006834
Test Image: subject02.asuka.jpg, Predicted Label: 2, Confidence: 51.351808634017296
Test Image: subject03.hori.jpg, Predicted Label: 3, Confidence: 44.75424378786359
Test Image: subject04.ikuta.jpg, Predicted Label: 4, Confidence: 45.70965779143789
Test Image: subject05shiraishi.jpg, Predicted Label: 5, Confidence: 49.08969544979982
Test Image: subject06nishino.jpg, Predicted Label: 6, Confidence: 43.72499865726888
Test Image: subject07.unagi.png, Predicted Label: 1, Confidence: 63.37511081588498
